In [1]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import socket, requests, json

In [2]:
# Spark Context configuration
sc = SparkContext("spark://fobidatalake02:7077", "NY")
spark = SparkSession(sc)

In [3]:
schema=StructType([StructField("address",StringType(),True),
StructField("attributes",StructType(),True),
StructField("business_id",StringType(),True),
StructField("city",StringType(),True),
StructField("hours",StructType(),True),
StructField("is_open",DoubleType(),True),
StructField("latitude",DoubleType(),True),
StructField("longitude",DoubleType(),True),
StructField("postal_code",StringType(),True),
StructField("review_count",LongType(),True),
StructField("stars",DoubleType(),True),
StructField("state",StringType(),True)])

In [4]:
biz = spark.read.json('file:///home/hadoopsa/data/biz.json', schema)
tip = spark.read.json('file:///home/hadoopsa/data/tip.json')
rvw = spark.read.json('file:///home/hadoopsa/data/review.json')
chk = spark.read.json('file:///home/hadoopsa/data/checkin.json')
usr = spark.read.json('file:///home/hadoopsa/data/user.json')

In [5]:
rvw = rvw.withColumn("DateTime_r", to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))
chk = chk.withColumn("DateTime_c", to_timestamp("date", "yyyy-MM-dd HH:mm:ss")).withColumnRenamed('business_id', 'business_id_chk')
rvwchk = rvw.join(chk, rvw.business_id == chk.business_id_chk, how ='left')

In [8]:
agg = rvwchk.groupBy("business_id", window(col("DateTime_r"), "168 hours").alias("Weekly")).agg(sum("stars").alias("stars"), count("business_id_chk").alias("checkins"))

In [9]:
agg.orderBy('stars', ascending=False).show(20, False)

+----------------------+------------------------------------------+-----+--------+
|business_id           |Weekly                                    |stars|checkins|
+----------------------+------------------------------------------+-----+--------+
|YZs1gNSh_sN8JmN_nrpxeA|[2019-06-27 02:00:00, 2019-07-04 02:00:00]|143.0|30      |
|bZiIIUcpgxh8mpKMDhdqbA|[2013-04-25 02:00:00, 2013-05-02 02:00:00]|137.0|31      |
|YZs1gNSh_sN8JmN_nrpxeA|[2019-08-15 02:00:00, 2019-08-22 02:00:00]|135.0|28      |
|YZs1gNSh_sN8JmN_nrpxeA|[2019-09-12 02:00:00, 2019-09-19 02:00:00]|134.0|28      |
|V6OToDLIwUllf1MbgWw-9w|[2017-11-02 01:00:00, 2017-11-09 01:00:00]|131.0|28      |
|bZiIIUcpgxh8mpKMDhdqbA|[2013-06-13 02:00:00, 2013-06-20 02:00:00]|122.0|29      |
|YZs1gNSh_sN8JmN_nrpxeA|[2019-09-26 02:00:00, 2019-10-03 02:00:00]|119.0|25      |
|bZiIIUcpgxh8mpKMDhdqbA|[2013-07-18 02:00:00, 2013-07-25 02:00:00]|119.0|27      |
|YZs1gNSh_sN8JmN_nrpxeA|[2019-05-23 02:00:00, 2019-05-30 02:00:00]|117.0|26      |
|YZs